## 0. Import All Libraries

In [1]:
from dotenv import load_dotenv
import os
from loguru import logger as LOGGER

from pydantic import BaseModel, Field
import pandas as pd
from tqdm import tqdm
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

load_dotenv("gpt-4o.env")
LOGGER.info(f"Current Model used: {os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')}")

2024-09-04 10:04:19.107 | INFO     | __main__:<module>:13 - Current Model used: gpt-4o


### 0.1 Global Variables

In [2]:
FP_HISTORICAL_DATA = "../historical_data/all_to_OMOP_Mapping.csv"

### 0.2 Add Support Functions

In [3]:
df_history = pd.read_csv(FP_HISTORICAL_DATA).head(268).dropna()


def provide_n_examples(n):
    """Add N Examples to the Prompt"""
    outputs = list(row.to_dict() for _, row in df_history.sample(n).iterrows())
    dict_ouptut = {}
    for idx, op in enumerate(outputs):
        dict_ouptut[f"example_src_table_{idx+1}"] = op["source_table"].lower()
        dict_ouptut[f"example_src_column_{idx+1}"] = op["source_column"].lower()
        dict_ouptut[f"example_trgt_table_{idx+1}"] = op["target_table"].lower()
        dict_ouptut[f"example_trgt_column_{idx+1}"] = op["target_column"].lower()
    return dict_ouptut


def gen_prompt_for_n_shots(n):
    input_vars = ["source_table", "source_column", "examples"]
    prompt_template = PromptTemplate(
        input_variables=input_vars,
        template="""
        You are a healthcare data expert agent who understands the dataware house ETL well and have an understanding on OMOP Common Data Model.
        You usually onboard new source tables on OMOP Data Set as you use this data for downstream Analytics and Apps to power your products. 
        Your job to provide data matching between any unknown source schema and OMOP table column. And you provide your best guess if you do not know the answer using chain of thoughts.

        Provide your answer in the following format:
        Target:
            Table: [OMOP table name]
            Column: [OMOP column name]

        {examples}

        Answer this Matching:
            Source: 
                Table: {source_table}
                Column: {source_column}
    """,
    )
    return prompt_template


def gen_n_examples(n):
    examples = []
    dict_output = provide_n_examples(n)
    for i in range(1, n + 1):
        src_table_key = f"example_src_table_{i}"
        src_colmn_key = f"example_src_column_{i}"
        tgt_table_key = f"example_trgt_table_{i}"
        tgt_colmn_key = f"example_trgt_column_{i}"
        examples.append(
            f"""
        Example {i}:
                Source: 
                    Table: {dict_output[src_table_key]}
                    Column: {dict_output[src_colmn_key]}
                Target:
                    Table: {dict_output[tgt_table_key]}
                    Column: {dict_output[tgt_colmn_key]}

        """
        )
    examples = "\n".join(examples)
    return examples


class MatchTarget(BaseModel):
    table: str = Field(
        description="The OMOP table that matches best with source schema table and column combination"
    )
    column: str = Field(
        description="The column corresponding to the OMOP table that matches best with source schema table and column combination"
    )


def process_csv(input_file, n_shots, chain):
    results = []

    reader = pd.read_csv(input_file)
    for _, row in tqdm(reader.iterrows()):
        source_table = row["source_table"]
        source_column = row["source_column"]
        examples = gen_n_examples(n_shots)
        input_map = {
            "source_table": source_table,
            "source_column": source_column,
            "examples": examples,
        }
        # Get prediction from the LLM
        response = chain.invoke(input_map)
        try:
            # Parse the response
            target_table = response.table
            target_column = response.column

        except:
            target_table = None
            target_column = None

        # Create the JSON object
        result = {
            "source_table": source_table,
            "source_column": source_column,
            "target_table_pred": target_table,
            "target_table_column_pred": target_column,
        }

        results.append(result)

    return results

## 1. Initialise Chain

In [4]:
# Initialize the OpenAI language model
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    openai_api_type="azure",
    temperature=0.1,
)

strucutred_llm = llm.with_structured_output(MatchTarget)

In [45]:
list_df_ops = list()
for n in range(1, 6):
    prompt_template = gen_prompt_for_n_shots(n)
    LOGGER.info(f"Chain Initiated || {n}-Shot Prompt(s) Matching using LLMs")
    # Create the LLMChain
    chain = prompt_template | strucutred_llm
    LOGGER.info(f"Execution Started || {n}-Shot Prompt(s) Matching using LLMs")
    op_results = process_csv(FP_HISTORICAL_DATA, n_shots=n, chain=chain)
    LOGGER.info(f"Execution Completed || {n}-Shot Prompt(s) Matching using LLMs")
    df_temp = pd.DataFrame(op_results)
    df_temp["n_shots"] = n
    list_df_ops.append(df_temp)

2024-09-04 08:28:49.632 | INFO     | __main__:<module>:4 - Chain Initiated || 1-Shot Prompt(s) Matching using LLMs
2024-09-04 08:28:49.633 | INFO     | __main__:<module>:7 - Execution Started || 1-Shot Prompt(s) Matching using LLMs
503it [05:25,  1.55it/s]
2024-09-04 08:34:15.050 | INFO     | __main__:<module>:9 - Execution Completed || 1-Shot Prompt(s) Matching using LLMs
2024-09-04 08:34:15.056 | INFO     | __main__:<module>:4 - Chain Initiated || 2-Shot Prompt(s) Matching using LLMs
2024-09-04 08:34:15.057 | INFO     | __main__:<module>:7 - Execution Started || 2-Shot Prompt(s) Matching using LLMs
503it [05:01,  1.67it/s]
2024-09-04 08:39:16.814 | INFO     | __main__:<module>:9 - Execution Completed || 2-Shot Prompt(s) Matching using LLMs
2024-09-04 08:39:16.818 | INFO     | __main__:<module>:4 - Chain Initiated || 3-Shot Prompt(s) Matching using LLMs
2024-09-04 08:39:16.819 | INFO     | __main__:<module>:7 - Execution Started || 3-Shot Prompt(s) Matching using LLMs
503it [05:22,  1

In [48]:
import asyncio
from langchain.callbacks import get_openai_callback


async def async_generate(chain, inputs):
    """
    Asynchronous task to extract sentiment and summary from a single review.
    Parameters
    ----------
    chain : SequentialChain
        The SequentialChain used for sentiment extraction.
    inputs : dict
        The inputs for the chain.
    unique_id : any
        The unique identifier for the review.
    Returns
    -------
    tuple
        A tuple containing the unique identifier, the extracted sentiment and summary, and the cost.
    """
    with get_openai_callback() as cb:
        resp = await chain.ainvoke(inputs)
    del inputs["examples"]
    return resp, inputs, cb.total_cost, cb.prompt_tokens, cb.completion_tokens


global_results = []


async def generate_concurrently(input_file, n_shots, chain):
    """
    Generates sentiment and summary concurrently for each review in the dataframe.
    The extracted sentiments, summaries, and costs are added to the dataframe.
    """
    tasks = []
    reader = pd.read_csv(input_file)
    for _, row in tqdm(reader.iterrows()):
        source_table = row["source_table"]
        source_column = row["source_column"]
        examples = gen_n_examples(n_shots)
        input_map = {
            "source_table": source_table,
            "source_column": source_column,
            "examples": examples,
        }
        tasks.append(async_generate(chain, input_map))

    results = await asyncio.gather(*tasks)
    global_results.extend(results)

In [49]:
def get_results(global_results):
    total_cost = sum([cost for record, input_value, cost, _, _ in global_results])
    list_ops = [record.dict() for record, input_value, cost, _, _ in global_results]
    list_ips = [input_value for record, input_value, cost, _, _ in global_results]
    total_prompt_tokens = sum([tokens for _, _, _, tokens, _ in global_results])
    total_completion_tokens = sum([tokens for _, _, _, _, tokens in global_results])
    df_temp_ops = pd.DataFrame(list_ops)
    df_temp_ops.columns = ["target_table_pred", "target_table_column_pred"]
    df_temp_ips = pd.DataFrame(list_ips)
    df_temp = pd.concat([df_temp_ips, df_temp_ops], axis=1)
    return df_temp, total_cost, total_prompt_tokens, total_completion_tokens

In [52]:
import time

MAX_SHOTS = 10
list_df_ops = []
overall_tokens_prompt = 0
overall_tokens_completion = 0
overall_cost = 0
for n_shot in range(1, MAX_SHOTS + 1):
    prompt_template = gen_prompt_for_n_shots(n_shot)
    LOGGER.info(f"Chain Initiated || {n_shot}-Shot Prompt(s) Matching using LLMs")
    # Create the LLMChain
    chain = prompt_template | strucutred_llm
    global_results = []
    LOGGER.info(
        f"Async Execution Started || {n_shot}-Shot Prompt(s) Matching using LLMs"
    )
    await generate_concurrently(FP_HISTORICAL_DATA, n_shot, chain)
    LOGGER.info(
        f"Async Execution Completed || {n_shot}-Shot Prompt(s) Matching using LLMs"
    )
    df_temp, total_cost, total_prompt_tokens, total_completion_tokens = get_results(
        global_results
    )
    LOGGER.info(f"Async Execution info for {n_shot}-Shot Prompt(s) Matching using LLMs")
    LOGGER.info(f"Total Cost: ${round(total_cost,3)}")
    LOGGER.info(f"Total Prompt Tokens: {round(total_prompt_tokens,3)}")
    LOGGER.info(f"Total Completion Tokens: {round(total_completion_tokens,3)}")
    df_temp["n_shots"] = n_shot
    list_df_ops.append(df_temp)
    overall_cost += total_cost
    overall_tokens_prompt += total_prompt_tokens
    overall_tokens_completion += total_completion_tokens
    if n_shot % 3 == 0:
        time.sleep(30)
    else:
        time.sleep(5)

LOGGER.info(f"Overall Cost: ${round(overall_cost,3)}")
LOGGER.info(f"Overall Prompt Tokens: {round(overall_tokens_prompt,3)}")
LOGGER.info(f"Overall Completion Tokens: {round(overall_tokens_completion,3)}")

2024-09-04 11:21:24.828 | INFO     | __main__:<module>:9 - Chain Initiated || 1-Shot Prompt(s) Matching using LLMs
2024-09-04 11:21:24.830 | INFO     | __main__:<module>:13 - Async Execution Started || 1-Shot Prompt(s) Matching using LLMs
503it [00:00, 4117.68it/s]
2024-09-04 11:21:30.195 | INFO     | __main__:<module>:15 - Async Execution Completed || 1-Shot Prompt(s) Matching using LLMs
2024-09-04 11:21:30.202 | INFO     | __main__:<module>:17 - Async Execution info for 1-Shot Prompt(s) Matching using LLMs
2024-09-04 11:21:30.203 | INFO     | __main__:<module>:18 - Total Cost: $0.785
2024-09-04 11:21:30.203 | INFO     | __main__:<module>:19 - Total Prompt Tokens: 137072
2024-09-04 11:21:30.204 | INFO     | __main__:<module>:20 - Total Completion Tokens: 6663
2024-09-04 11:21:35.210 | INFO     | __main__:<module>:9 - Chain Initiated || 2-Shot Prompt(s) Matching using LLMs
2024-09-04 11:21:35.210 | INFO     | __main__:<module>:13 - Async Execution Started || 2-Shot Prompt(s) Matching u

In [53]:
df_results = pd.concat(list_df_ops)
df_results.to_csv("few_shot_ops_gpt_4o.csv", index=False)

In [54]:
df_results

,source_table,source_column,target_table_pred,target_table_column_pred,n_shots
0,ADMISSIONS,SUBJECT_ID,person,person_id,1
1,ADMISSIONS,HADM_ID,visit_occurrence,visit_occurrence_id,1
2,ADMISSIONS,ADMITTIME,visit_occurrence,visit_start_datetime,1
3,ADMISSIONS,DISCHTIME,visit_occurrence,visit_end_datetime,1
4,ADMISSIONS,DEATHTIME,death,death_datetime,1
...,...,...,...,...,...
498,Pharmacy Claims,NDC,drug_exposure,drug_source_value,10
499,Pharmacy Claims,SPCLT_IND,drug_exposure,specialty_concept_id,10
500,Pharmacy Claims,HCCI_HL_CAT,drug_exposure,drug_type_concept_id,10
501,Pharmacy Claims,HCCI_DET_CAT,drug_exposure,drug_type_concept_id,10
